In [3]:
##Streamlit - Desempenho de Vendas

In [4]:
import pandas as pd

In [7]:
df_test = pd.read_csv('C:/Users/wagne/IntelliJ Idea-Workspace/Desempenho-de-vendas/Database_CSV/Test.csv')
df_test.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3
5,FDH56,9.800,Regular,0.063817,Fruits and Vegetables,117.1492,OUT046,1997,Small,Tier 1,Supermarket Type1
6,FDL48,19.350,Regular,0.082602,Baking Goods,50.1034,OUT018,2009,Medium,Tier 3,Supermarket Type2
7,FDC48,NaN,Low Fat,0.015782,Baking Goods,81.0592,OUT027,1985,Medium,Tier 3,Supermarket Type3
8,FDN33,6.305,Regular,0.123365,Snack Foods,95.7436,OUT045,2002,NaN,Tier 2,Supermarket Type1
9,FDA36,5.985,Low Fat,0.005698,Baking Goods,186.8924,OUT017,2007,NaN,Tier 2,Supermarket Type1


In [9]:
df_train = pd.read_csv('C:/Users/wagne/IntelliJ Idea-Workspace/Desempenho-de-vendas/Database_CSV/Train.csv')
df_train.head(10)

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.300,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.920,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.500,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.200,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.930,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052
5,FDP36,10.395,Regular,0.000000,Baking Goods,51.4008,OUT018,2009,Medium,Tier 3,Supermarket Type2,556.6088
6,FDO10,13.650,Regular,0.012741,Snack Foods,57.6588,OUT013,1987,High,Tier 3,Supermarket Type1,343.5528
7,FDP10,NaN,Low Fat,0.127470,Snack Foods,107.7622,OUT027,1985,Medium,Tier 3,Supermarket Type3,4022.7636
8,FDH17,16.200,Regular,0.016687,Frozen Foods,96.9726,OUT045,2002,NaN,Tier 2,Supermarket Type1,1076.5986
9,FDU28,19.200,Regular,0.094450,Frozen Foods,187.8214,OUT017,2007,NaN,Tier 2,Supermarket Type1,4710.5350


In [11]:
df_test = df_test[["Outlet_Type", "Outlet_Size", "Outlet_Location_Type", "Item_Visibility"]]
df_train = df_train[["Outlet_Type", "Outlet_Size", "Outlet_Location_Type", "Item_Visibility"]]

In [14]:
df_test.head(10)

,Outlet_Type,Outlet_Size,Outlet_Location_Type,Item_Visibility
0,Supermarket Type1,Medium,Tier 1,0.007565
1,Supermarket Type1,NaN,Tier 2,0.038428
2,Grocery Store,NaN,Tier 3,0.099575
3,Supermarket Type1,NaN,Tier 2,0.015388
4,Supermarket Type3,Medium,Tier 3,0.118599
5,Supermarket Type1,Small,Tier 1,0.063817
6,Supermarket Type2,Medium,Tier 3,0.082602
7,Supermarket Type3,Medium,Tier 3,0.015782
8,Supermarket Type1,NaN,Tier 2,0.123365
9,Supermarket Type1,NaN,Tier 2,0.005698


In [16]:

df_train.head(10)

,Outlet_Type,Outlet_Size,Outlet_Location_Type,Item_Visibility
0,Supermarket Type1,Medium,Tier 1,0.016047
1,Supermarket Type2,Medium,Tier 3,0.019278
2,Supermarket Type1,Medium,Tier 1,0.016760
3,Grocery Store,NaN,Tier 3,0.000000
4,Supermarket Type1,High,Tier 3,0.000000
5,Supermarket Type2,Medium,Tier 3,0.000000
6,Supermarket Type1,High,Tier 3,0.012741
7,Supermarket Type3,Medium,Tier 3,0.127470
8,Supermarket Type1,NaN,Tier 2,0.016687
9,Supermarket Type1,NaN,Tier 2,0.094450


In [20]:
df_train_Proc = pd.get_dummies(df_train, columns=['Outlet_Type', 'Outlet_Size', 'Outlet_Location_Type'], dtype=int)
df_train_Proc.head(10)

,Item_Visibility,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3
0,0.016047,0,1,0,0,0,1,0,1,0,0
1,0.019278,0,0,1,0,0,1,0,0,0,1
2,0.016760,0,1,0,0,0,1,0,1,0,0
3,0.000000,1,0,0,0,0,0,0,0,0,1
4,0.000000,0,1,0,0,1,0,0,0,0,1
5,0.000000,0,0,1,0,0,1,0,0,0,1
6,0.012741,0,1,0,0,1,0,0,0,0,1
7,0.127470,0,0,0,1,0,1,0,0,0,1
8,0.016687,0,1,0,0,0,0,0,0,1,0
9,0.094450,0,1,0,0,0,0,0,0,1,0


In [25]:
features_Test = ['Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3','Outlet_Size_High',
                 'Outlet_Size_Medium', 'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3']
print(features_Test)

['Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3', 'Outlet_Size_High', 'Outlet_Size_Medium', 'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3']


In [26]:
Test_X = df_test_Proc[features_Test]
Test_X.head(10)

,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3
0,0,1,0,0,0,1,0,1,0,0
1,0,1,0,0,0,0,0,0,1,0
2,1,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,1,0
4,0,0,0,1,0,1,0,0,0,1
5,0,1,0,0,0,0,1,1,0,0
6,0,0,1,0,0,1,0,0,0,1
7,0,0,0,1,0,1,0,0,0,1
8,0,1,0,0,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,1,0


In [27]:
features_Train = ['Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3','Outlet_Size_High',
                 'Outlet_Size_Medium', 'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3']
print(features_Train)

['Outlet_Type_Grocery Store', 'Outlet_Type_Supermarket Type1', 'Outlet_Type_Supermarket Type2', 'Outlet_Type_Supermarket Type3', 'Outlet_Size_High', 'Outlet_Size_Medium', 'Outlet_Size_Small', 'Outlet_Location_Type_Tier 1', 'Outlet_Location_Type_Tier 2', 'Outlet_Location_Type_Tier 3']


In [28]:
Train_X = df_train_Proc[features_Train]
Train_X.head(10)

,Outlet_Type_Grocery Store,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3,Outlet_Size_High,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Location_Type_Tier 1,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3
0,0,1,0,0,0,1,0,1,0,0
1,0,0,1,0,0,1,0,0,0,1
2,0,1,0,0,0,1,0,1,0,0
3,1,0,0,0,0,0,0,0,0,1
4,0,1,0,0,1,0,0,0,0,1
5,0,0,1,0,0,1,0,0,0,1
6,0,1,0,0,1,0,0,0,0,1
7,0,0,0,1,0,1,0,0,0,1
8,0,1,0,0,0,0,0,0,1,0
9,0,1,0,0,0,0,0,0,1,0


In [39]:
Test_Y = df_test_Proc['Item_Visibility']
print(Test_Y.head())


0    0.007565
1    0.038428
2    0.099575
3    0.015388
4    0.118599
Name: Item_Visibility, dtype: float64


In [42]:
Train_Y = df_train_Proc['Item_Visibility']
print(Train_Y.head())

0    0.016047
1    0.019278
2    0.016760
3    0.000000
4    0.000000
Name: Item_Visibility, dtype: float64
